In [1]:
#importando bibliotecas
from IPython.display import clear_output
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd

#Obtendo o HTML principal
url = 'https://www.ludopedia.com.br/search_jogo?advsearch=true&tipo=BG&fl_tp_jogo=1&s=1&pagina=1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0'}

try:
    req = Request(url, headers = headers) #faz o request com o url e informacao do agente usuario navegador
    response = urlopen(req)  #abre a pagina

    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

except HTTPError as e:
    print(e.status, e.reason) #captura erro de requisicao
    
except URLError as e:
    print(e.reason)  #captura erro de URL

In [ ]:
#Declarando váriavel jogos
jogos = []


#Obtendo paginas da busca
pages =  int(soup.find('a', title = 'Última Página').get('href').split('=')[-1])
pages

In [ ]:
#Iterando todas as paginas do site
contador = 0

for i in range(pages):
    
    
    #Obtendo HTML de cada pagina de resultado
    url = 'https://www.ludopedia.com.br/search_jogo?advsearch=true&tipo=BG&fl_tp_jogo=1&s=1&pagina='+ str(i + 564)
    
    try:
        req = Request(url, headers = headers) #faz o request com o url e informacao do agente usuario navegador
        response = urlopen(req)  #abre a pagina

        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason) #captura erro de requisicao

    except URLError as e:
        print(e.reason)  #captura erro de URL
    
    #Entrando no link de cada jogo
    
    links = soup.findAll('a', class_ = "full-link")
    for link in links:
        response = urlopen(link.get('href'))
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser') 
        
        #criar variavel jogo
        jogo = {}
        
        #Extrair Nome
        jogo['Nome'] = soup.find('div', class_ = 'jogo-top-main').find('a', href = '#').getText()
        
        
        #Funçao para extrair Designer, Artista e Editora
        atributo = {'Designer': 0, 'Artista': 1, 'Editora': 2}

        def listaDados(dado):
            try:
                jogo[dado] = soup.find('div', class_ = 'jogo-top-main').findAll('span', class_ = 'info-span text-sm')[atributo[dado]].find('a').getText()
            except:
                jogo[dado] = None

        listaDados('Designer')
        listaDados('Artista')
        listaDados('Editora')
        
        
        #Extrair Domínio
        listaDominio = soup.find('div', class_ = 'mar-btm bg-gray-light pad-all').find('h4', string = 'Domínios').findNextSiblings()
        for d in listaDominio:
            if d.name == "h4":
                break
            else:
                jogo['Domínio'] = d.getText()
        
        
        #Funcao para extrair Mecanicas, Categorias e Temas
        def listaItens(item):
            itens=[]
            lista = soup.find('div', class_ = 'mar-btm bg-gray-light pad-all').find('h4', string = item).findNextSiblings()
            for d in lista:
                if d.name == "h4":
                    break
                else:
                    itens.append(d.getText())

            jogo[item] = itens
            
        listaItens('Mecânicas')
        listaItens('Categorias')
        listaItens('Temas')
        
        #Extrair Descriçao
        jogo['Descrição'] = soup.find('div', id = 'bloco-descricao-sm').p.getText().replace('\n\r\n', ' ').replace('\r\n', ' ').replace('*', '')
        
        #Adicionar jogo na lista de jogos
        jogos.append(jogo)
        
        #contador loading
        clear_output(wait = True)
        contador += 1
        print('Extraindo jogo ' + str(contador))

In [ ]:
datasetLudopedia = pd.DataFrame(jogos)
datasetLudopedia.to_csv('./dataset/datasetLudopedia.csv', sep = ',', index = False, encoding= 'utf-8-sig')
datasetLudopedia